<a href="https://colab.research.google.com/github/BeckieHuang/WhetherRain/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install sklearn
import pandas as pd
import numpy as np

# 繪圖相關套件
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec
import seaborn as sns
plt.style.use( 'ggplot' ) 

# 標籤編碼(Label)、獨熱編碼(OneHot)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
# 決策樹(Decision Tree)
from sklearn.tree import DecisionTreeClassifier
# 隨機森林(Random Forest)
from sklearn.ensemble import RandomForestClassifier

from IPython.display import display
import warnings
warnings.filterwarnings( 'ignore' )

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
submit = pd.read_csv('ex_submit.csv')

print( f'train {df_train.shape}' )
display( df_train.head() )

print( f'\ntest {df_test.shape}' )
display( df_test.head() )

train (17093, 23)


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22,Attribute23
0,2008-12-01,2,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-17,2,14.1,20.9,0.0,NaN,NaN,ENE,22.0,SSW,E,11.0,9.0,69.0,82.0,1012.2,1010.4,8.0,1.0,17.2,18.1,No,Yes
2,2008-12-20,2,9.8,25.6,0.0,NaN,NaN,SSE,26.0,SE,NNW,17.0,6.0,45.0,26.0,1019.2,1017.1,NaN,NaN,15.8,23.2,No,No
3,2008-12-30,2,12.5,24.2,1.2,NaN,NaN,WNW,50.0,WSW,SW,11.0,22.0,78.0,70.0,1005.6,1003.4,8.0,8.0,12.5,18.2,Yes,No
4,2009-01-22,2,24.4,34.0,0.6,NaN,NaN,NW,98.0,N,NNW,26.0,48.0,60.0,39.0,1005.3,997.8,4.0,1.0,26.1,33.1,No,Yes



test (790, 22)


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22
0,2009-01-02,10,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No
1,2009-07-26,10,8.9,12.2,0.8,2.4,1.2,N,24.0,NNE,S,7.0,11.0,91.0,77.0,1018.1,1013.5,8.0,7.0,9.1,11.4,No
2,2010-01-19,10,11.3,28.8,0.0,11.2,12.9,SSW,50.0,S,SSW,9.0,20.0,29.0,12.0,1012.9,1011.2,0.0,0.0,18.1,26.5,No
3,2010-03-03,10,15.8,30.4,0.0,8.0,6.3,E,39.0,ESE,E,22.0,20.0,59.0,30.0,1013.7,1010.5,1.0,5.0,19.0,29.4,No
4,2010-06-25,10,10.0,22.0,0.0,3.0,7.7,NNW,39.0,NNE,NW,20.0,19.0,73.0,52.0,1022.3,1019.0,1.0,6.0,13.7,20.8,No


In [ ]:
df_train.dropna(inplace=True)

In [ ]:
# 定義用來統計欄位缺漏值總數的函數
def Missing_Counts( Data ) : 
    missing = Data.isnull().sum()  # 計算欄位中缺漏值的數量 
    missing = missing[ missing>0 ]
    missing.sort_values( inplace=True ) 
    
    Missing_Count = pd.DataFrame( { 'ColumnName':missing.index, 'MissingCount':missing.values } )  # Convert Series to DataFrame
    Missing_Count[ 'Percentage(%)' ] = Missing_Count['MissingCount'].apply( lambda x:round(x/Data.shape[0]*100,2) )
    return  Missing_Count

In [ ]:
print( 'train :' )
display( Missing_Counts(df_train) )

print( 'test :' )
display( Missing_Counts(df_test) )

train :


,ColumnName,MissingCount,Percentage(%)


test :


,ColumnName,MissingCount,Percentage(%)


In [ ]:
df_data = df_train.append( df_test )
df_data

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22,Attribute23
706,2009-01-04,10,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
707,2009-01-07,10,27.1,36.1,0.0,13.0,0.0,N,43.0,N,WNW,7.0,20.0,26.0,19.0,1007.7,1007.4,8.0,8.0,30.7,34.3,No,No
708,2009-01-08,10,23.3,34.0,0.0,9.8,12.6,SSW,41.0,S,SSE,17.0,19.0,33.0,15.0,1011.3,1009.9,3.0,1.0,25.0,31.5,No,No
709,2009-01-18,10,17.9,34.2,0.0,12.0,11.3,SE,61.0,NE,SE,22.0,17.0,46.0,19.0,1016.4,1013.5,1.0,2.0,21.6,32.2,No,No
710,2009-01-21,10,25.4,33.5,0.0,13.6,3.7,N,46.0,NW,N,9.0,28.0,46.0,52.0,1012.0,1009.8,4.0,7.0,28.9,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,2016-04-22,13,24.9,35.9,0.0,6.0,11.0,E,41.0,E,SE,11.0,22.0,68.0,26.0,1012.6,1009.0,1.0,4.0,30.1,34.8,No,NaN
786,2016-04-30,13,25.2,34.2,0.0,4.4,8.0,ENE,31.0,S,NW,9.0,17.0,75.0,62.0,1012.8,1008.6,6.0,7.0,29.3,32.0,No,NaN
787,2016-11-06,13,22.7,31.4,25.6,7.0,3.4,SSE,41.0,NNW,NNW,7.0,11.0,79.0,93.0,1011.9,1008.5,7.0,7.0,30.4,25.9,Yes,NaN
788,2016-12-26,13,25.4,33.4,0.0,4.8,6.3,W,52.0,ENE,WNW,7.0,13.0,85.0,55.0,1006.6,1004.5,8.0,7.0,28.0,32.3,No,NaN


In [ ]:
df_data['Attribute22'].replace({'No':0,'Yes':1},inplace=True)
df_data['Attribute23'].replace({'No':0,'Yes':1},inplace=True)

In [ ]:
for col in ['Attribute8','Attribute10','Attribute11']:
    df_data[col] = df_data[col].astype('category').cat.codes

In [ ]:
df_data.shape
df_data.head()
display( Missing_Counts(df_data) )

,ColumnName,MissingCount,Percentage(%)
0,Attribute23,790,10.36


In [ ]:
# 產生訓練集和測試集
Train = df_data[ pd.notnull(df_data.Attribute23) ]
Test = df_data[ pd.isnull(df_data.Attribute23) ]


Train.drop( ['Attribute1','Attribute13','Attribute2','Attribute12','Attribute19','Attribute6','Attribute22','Attribute18','Attribute10','Attribute11','Attribute8'], axis=1, inplace=True )
Test.drop( ['Attribute1','Attribute23','Attribute13','Attribute2','Attribute12','Attribute19','Attribute6','Attribute22','Attribute18','Attribute10','Attribute11','Attribute8'], axis=1, inplace=True )

# 將測試集中的標籤欄位 Survived 單獨拆出
Y_Train = Train.Attribute23
X_Train = Train.drop( ['Attribute23'], axis=1 )

print( f'Shape of X_Train = {X_Train.shape}' )
print( f'Shape of Y_Train = {Y_Train.shape}' )
print( f'Shape of Test = {Test.shape}' )

Shape of X_Train = (6838, 11)
Shape of Y_Train = (6838,)
Shape of Test = (790, 11)


In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X_Train.values, Y_Train.values, test_size=0.25,random_state=89)

my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [ ]:
# from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

from xgboost import XGBRegressor
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100,max_depth=4, 
                learning_rate= 0.06,
                min_child_weight=1,scale_pos_weight = 6,colsample_bytree=0.5,random_state=89)
# 使用訓練資料訓練模型
xgboostModel.fit(train_X, train_y)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(test_X)

# # xgb_model = XGBRegressor(nthread=7,random_state=89)
# cv_split = ShuffleSplit(n_splits=6, train_size=0.75, test_size=0.25)
# grid_params = dict(
#     max_depth = [4, 5, 6, 7],
#     learning_rate = np.linspace(0.03, 0.3, 10),
#     n_estimators = [100, 200],
#     scale_pos_weight = np.linspace(1, 8,2),
#     colsample_bytree = 
# )
# grid = GridSearchCV(xgboostModel, grid_params, cv=cv_split, scoring='neg_mean_squared_error')
# grid.fit(train_X, train_y)


# # # 建立 XGBRegressor 模型

# # my_model = XGBRegressor(learning_rate =0.1, n_estimators=30, max_depth=12, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
# # # Add silent=True to avoid printing out updates with each cycle
# # my_model.fit(train_X, train_y)
# # predicted = my_model.predict(test_X)


# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score

# features = ['Attribute15']

# param_test = {
#     'n_estimators': range(30, 1000, 100),
#     'max_depth': range(2, 12, 2),
#     'min_child_weight':range(1,7,2)
# }
# grid_search = GridSearchCV(estimator = my_model, param_grid = param_test, 
# scoring='accuracy', cv=5)
# grid_search.fit(Train[features], Train['Attribute23'])
# means = grid_search.cv_results_['mean_test_score']
# grid_search.best_params_, grid_search.best_score_





In [ ]:
# print(grid.best_params_,grid.best_score_)
# # print(xgboostModel.feature_importances_)
# print(grid.best_params_)
# print('rmse:', (grid.best_score_) ** 0.5)

In [ ]:
# from sklearn.metrics import mean_absolute_error
# print("Mean Absolute Error : " + str(mean_absolute_error(predicted, test_y)))
print('訓練集: ',xgboostModel.score(train_X,train_y))
print('測試集: ',xgboostModel.score(test_X,test_y))

# print('訓練集: ',my_model.score(train_X,train_y))
# print('測試集: ',my_model.score(test_X,test_y))


訓練集:  0.8116224648985959
測試集:  0.8


In [ ]:
Test_TEST_X = my_imputer.transform(Test)
Test

,Attribute3,Attribute4,Attribute5,Attribute7,Attribute9,Attribute14,Attribute15,Attribute16,Attribute17,Attribute20,Attribute21
0,18.4,28.9,0.0,13.0,37.0,30.0,8.0,1012.9,1012.1,20.3,27.0
1,8.9,12.2,0.8,1.2,24.0,91.0,77.0,1018.1,1013.5,9.1,11.4
2,11.3,28.8,0.0,12.9,50.0,29.0,12.0,1012.9,1011.2,18.1,26.5
3,15.8,30.4,0.0,6.3,39.0,59.0,30.0,1013.7,1010.5,19.0,29.4
4,10.0,22.0,0.0,7.7,39.0,73.0,52.0,1022.3,1019.0,13.7,20.8
...,...,...,...,...,...,...,...,...,...,...,...
785,24.9,35.9,0.0,11.0,41.0,68.0,26.0,1012.6,1009.0,30.1,34.8
786,25.2,34.2,0.0,8.0,31.0,75.0,62.0,1012.8,1008.6,29.3,32.0
787,22.7,31.4,25.6,3.4,41.0,79.0,93.0,1011.9,1008.5,30.4,25.9
788,25.4,33.4,0.0,6.3,52.0,85.0,55.0,1006.6,1004.5,28.0,32.3


In [ ]:
# new_predicted = grid.predict(Test_TEST_X)


In [ ]:
# new_predicted = my_model.predict(Test_TEST_X)
new_predicted = xgboostModel.predict(Test_TEST_X)
new_predicted
# new_predicted.round().astype(int)

array([0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 1.

In [ ]:
submit['ans'] = new_predicted.astype(int)
submit.to_csv( 'ans.csv', index=False )
print( f'預測結果：' )
submit 

預測結果：


,id,ans
0,0.0,0
1,1.0,1
2,2.0,0
3,3.0,0
4,4.0,0
...,...,...
785,785.0,0
786,786.0,1
787,787.0,1
788,788.0,1
